In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import pandas as pd
books = pd.read_csv("books_cleaned.csv")

In [6]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep="\n",
                                   index=False,
                                   header=False,
                                    encoding="utf-8")

In [7]:
raw_documents=TextLoader("tagged_description.txt",encoding="utf-8").load()
text_splitter =  CharacterTextSplitter(chunk_size=100000,chunk_overlap=0,separator="\n")
documents=text_splitter.split_documents(raw_documents)

In [8]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [14]:
# pip install -U langchain-huggingface sentence-transformers

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"},                 # or "cuda"
    encode_kwargs={"normalize_embeddings": True}
)

db_books = Chroma.from_documents(documents, embedding=embedding,
                                 persist_directory="./chroma_mpnet")



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Nikhil\Desktop\bookrecommender\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nikhil\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
query = "A book to teach children about nature"
docs=db_books.similarity_search(query,k=10)
docs

[Document(id='4c5b3081-cc14-48af-a395-e2f5f0541d52', metadata={'source': 'tagged_description.txt'}, page_content='9780440410256 Thirteen-year-old Nikki Roberts tries to help two children trapped by a forest fire but finds her efforts blocked by poachers who want her to become one of the fire\'s victims. But after a shocking turn of events, the fire becomes the least of Nikki\'s problems. Will Nikki\'s horse, Goblin, and her years of hunting guide experience save her, or will Fire Mountain reign supreme?\n9780440412243 Abandoned as babies on the doorstep of television star, Malcolm Masters, Thomas, Lucy, Margery, and William also become TV performers, but problems arise when Malcolm goes off on an extended cruise, leaving the children behind.\n9780440414001 Sam Krupnik, mad at his mother because she won\'t let him wear his new plastic fangs in the house, decides to run away to Alaska.\n9780440415244 Separated early in life from his mother and siblings, Keeper tells of his adventures wit

In [16]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,titles_and_subtitle,tagged_description
2203,9780440410256,0440410258,Escape from Fire Mountain,Gary Paulsen,Juvenile Fiction,http://books.google.com/books/content?id=jvhtD...,Thirteen-year-old Nikki Roberts tries to help ...,1995.0,3.7,80.0,112.0,Escape from Fire Mountain,9780440410256 Thirteen-year-old Nikki Roberts ...


In [21]:

def retrieve_semantic_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
    """
    Given a query string, return the top_k most semantically similar books.
    """
    # Search in your Chroma DB
    recs = db_books.similarity_search(query, k=50)  # fetch more to refine
    
    # Extract ISBNs from search results (they are at the start of page_content)
    books_list = []
    for i in range(0,len(recs)):
        books_list +=[int(recs[i].page_content.strip('"').split()[0])]
    
    # Filter your original DataFrame
    return books[books["isbn13"].isin(books_list)].head(top_k)


In [22]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,titles_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
156,9780060574215,0060574216,"Men Are from Mars, Women Are from Venus",John Gray,Family & Relationships,http://books.google.com/books/content?id=MUw_d...,Rediscover the most famous relationship book e...,2004.0,3.54,368.0,126108.0,"Men Are from Mars, Women Are from Venus: The C...",9780060574215 Rediscover the most famous relat...
293,9780060927196,0060927194,Success Stories,Russell Banks,Fiction,http://books.google.com/books/content?id=bGIs7...,"In Sucess Stories, an exceptionally varied yet...",1996.0,3.72,192.0,148.0,Success Stories,"9780060927196 In Sucess Stories, an exceptiona..."
446,9780071418584,007141858X,How to Talk to Anyone : 92 Little Tricks for B...,Leil Lowndes,Family & Relationships,http://books.google.com/books/content?id=LQt6m...,"""You'll not only break the ice, you'll melt it...",2003.0,3.78,345.0,15488.0,How to Talk to Anyone : 92 Little Tricks for B...,"9780071418584 ""You'll not only break the ice, ..."
638,9780140299144,0140299149,Vinyl Cafe Unplugged,Stuart McLean,Canadian wit and humor,http://books.google.com/books/content?id=FQjXM...,Bestselling author and radio storytelling sens...,2001.0,4.23,272.0,1295.0,Vinyl Cafe Unplugged,9780140299144 Bestselling author and radio sto...
856,9780143037439,0143037439,Tooth and Claw,T. Coraghessan Boyle,Fiction,http://books.google.com/books/content?id=ebK4H...,"A collection of fourteen stories includes ""The...",2006.0,3.82,304.0,1165.0,Tooth and Claw,9780143037439 A collection of fourteen stories...
1036,9780226042756,0226042758,Plato's Symposium,Plato;Seth Benardete,Philosophy,http://books.google.com/books/content?id=0zoQD...,"Plato, Allan Bloom wrote, is ""the most erotic ...",2001.0,4.03,199.0,501.0,Plato's Symposium: A Translation by Seth Benar...,"9780226042756 Plato, Allan Bloom wrote, is ""th..."
1201,9780312267179,0312267177,Catfish and Mandala,Andrew X. Pham,Biography & Autobiography,http://books.google.com/books/content?id=GMqwB...,A Vietnamese American returns to the land of h...,2000.0,3.96,344.0,4541.0,Catfish and Mandala: A Two-Wheeled Voyage Thro...,9780312267179 A Vietnamese American returns to...
1391,9780330351690,0330351699,Into the Wild,Jon Krakauer,True Crime,http://books.google.com/books/content?id=X7M1D...,"Using the true story of a young man, who in 19...",1998.0,3.97,206.0,1268.0,Into the Wild,9780330351690 Using the true story of a young ...
1618,9780373771721,037377172X,Call After Midnight & Under the Knife,Tess Gerritsen,Fiction,http://books.google.com/books/content?id=bqw__...,A ringing phone in the middle of the night sha...,2006.0,4.00,555.0,244.0,Call After Midnight & Under the Knife,9780373771721 A ringing phone in the middle of...
